In [1]:
from src.core.gamma_client import GammaMarketsClient
from src.execution.order_executor import OrderExecutor
from src.core.clob_client import PolymarketClient
from py_clob_client.clob_types import ApiCreds, OrderArgs

gamma_client = GammaMarketsClient()

# Fetch active events with high liquidity and volume
markets = gamma_client.get_markets(
    closed=False,  # Exclude closed markets
    liquidity_num_min=60000.0,  # Minimum liquidity
    volume_num_min=5000.0,  # Minimum trading volume
    start_date_min="2025-03-10",  # Markets starting after this date
    tag_id=1
    
)

markets_key = gamma_client.filter_markets_by_slug_keyword(markets=markets, keyword="nba")

if markets_key:
    print(f"Total markets found: {len(markets_key)}")
    for market in markets_key:
        # Safely access keys using .get() to avoid KeyError
        question = market.get("question", "N/A")
        event_id = market.get("id", "N/A")
        volume = market.get("volume", "N/A")  # Use .get() to handle missing keys
        active = market.get("active", "N/A")
        closed = market.get("closed", "N/A")
        clobTokenIds = market.get("clobTokenIds")
        orderBook = market.get("enableOrderBook", "N/A")
        print(f"Question: {question}, Event ID: {event_id}, Volume: {volume}, Active: {active}, Enabled Order book: {orderBook}")
        print(f"clobTokenIds  {clobTokenIds},")
else:
    print("No markets found matching the criteria.")

Found 11 markets with 'nba' in slug
Total markets found: 11
Question: Rockets vs. Magic, Event ID: 528363, Volume: 165988.185223, Active: True, Enabled Order book: True
clobTokenIds  ["43186308813322774389511655009441265359358460001746875331763940432725323754100", "42475543248404210065990230898512565125915441850041772863138114442518722564349"],
Question: Mavericks vs. Pacers, Event ID: 528364, Volume: 118903.072778, Active: True, Enabled Order book: True
clobTokenIds  ["21666793519107110553825112963696295201139871889688131710582076581472267702286", "84110361824231682481656273790222065034367107919161812176865639635357181855701"],
Question: Pistons vs. Heat, Event ID: 528365, Volume: 69614.639507, Active: True, Enabled Order book: True
clobTokenIds  ["85904339173069230948579711819008479036336171745116659010524049782494847727992", "47430731082598180016768281342526340602096563589309714063079380071748556210795"],
Question: Pelicans vs. Timberwolves, Event ID: 528366, Volume: 219655.551431, 

In [4]:
# Question: Will Arsenal finish in 2nd place for the 2024-25 EPL season?
yes_token= "40465031556608279901803453704591823162810651299818097010002629371896228132129"

client = PolymarketClient()

# Retrieve order book information
orderbook = client.get_order_book(yes_token)
midpoint = client.get_midpoint_price(yes_token)
best_buy = client.get_price(yes_token, "BUY")
best_sell = client.get_price(yes_token, "SELL")
spread = client.get_spread(yes_token)


print("Order Book:")
print(orderbook)
print("\nMarket Information:")
print(f"Midpoint Price: {midpoint}")
print(f"Best Buy Price: {best_buy}")
print(f"Best Sell Price: {best_sell}")
print(f"Spread: {spread}")

PolymarketClient initialized with address: 0xA8fdCA71fa876ab477B3b5d4725292FFDacCd04F
Order Book:
OrderBookSummary(market='0xcb3906fe1cc346a08d4817a9e5abbd594d50101d2d70edf92dceeeed509a328d', asset_id='62697312879578878537492465609249634498018844363287127652537828808816942160117', timestamp='1742070112627', bids=[OrderSummary(price='0.01', size='557305'), OrderSummary(price='0.02', size='200000'), OrderSummary(price='0.03', size='23338'), OrderSummary(price='0.04', size='22222'), OrderSummary(price='0.05', size='50000'), OrderSummary(price='0.08', size='1250'), OrderSummary(price='0.1', size='150'), OrderSummary(price='0.13', size='400'), OrderSummary(price='0.14', size='1000'), OrderSummary(price='0.17', size='589'), OrderSummary(price='0.18', size='1000'), OrderSummary(price='0.35', size='286'), OrderSummary(price='0.36', size='300'), OrderSummary(price='0.37', size='990'), OrderSummary(price='0.41', size='2444'), OrderSummary(price='0.42', size='900'), OrderSummary(price='0.6', size

In [9]:
from py_clob_client.order_builder.constants import BUY, SELL


# Instantiate the order executor
executor = OrderExecutor()

# Define a signal for a limit order.
# In a real strategy, this signal would be generated dynamically.
signal = {
    "token_id": yes_token,
    "order_type": "limit",
    "side": "BUY",
    "price": 0.10,      
    "quantity": 5.0   
}




PolymarketClient initialized with address: 0xA8fdCA71fa876ab477B3b5d4725292FFDacCd04F


In [11]:


response = executor.execute_signal(signal)
print("\nLimit Order Response:")
print(response)

Placing LIMIT order: BUY 5.0 of 62697312879578878537492465609249634498018844363287127652537828808816942160117 at 0.1

Limit Order Response:
{'errorMsg': '', 'orderID': '0x0714c38fca8239fa52ef1e3f3a68ef257141921db81df9abe801abb233e77e54', 'takingAmount': '', 'makingAmount': '', 'status': 'live', 'transactionsHashes': None, 'success': True}


In [13]:
# If the order_response contains an order ID, cancel the order.
order_id = response.get("orderId") if isinstance(response, dict) else None
executor.cancel_all_orders()
if order_id:
    print(f"\nOrder placed with ID: {order_id}. Now cancelling...")
    cancel_response = executor.cancel_order(order_id)
    print("Cancel Order Response:")
    print(cancel_response)
else:
    print("\nNo order ID found in response; cannot cancel the order.")

Cancelling all orders.

No order ID found in response; cannot cancel the order.


In [12]:
# Check orders
print("\nOpen Orders:")
print(executor.client.get_open_orders())



Open Orders:
[{'id': '0x0714c38fca8239fa52ef1e3f3a68ef257141921db81df9abe801abb233e77e54', 'status': 'LIVE', 'owner': '7bb51f33-4d47-0d39-b420-f933c332384f', 'maker_address': '0x6979603783972524560F94b1624E147C3435D7a2', 'market': '0xcb3906fe1cc346a08d4817a9e5abbd594d50101d2d70edf92dceeeed509a328d', 'asset_id': '62697312879578878537492465609249634498018844363287127652537828808816942160117', 'side': 'BUY', 'original_size': '5', 'size_matched': '0', 'price': '0.1', 'outcome': 'Celtics', 'expiration': '0', 'order_type': 'GTC', 'associate_trades': [], 'created_at': 1742070537}, {'id': '0x861e11be765df15f6862383dcdf9fd40fb7cf340b68c7b77e0bd659030b38725', 'status': 'LIVE', 'owner': '7bb51f33-4d47-0d39-b420-f933c332384f', 'maker_address': '0x6979603783972524560F94b1624E147C3435D7a2', 'market': '0xcb3906fe1cc346a08d4817a9e5abbd594d50101d2d70edf92dceeeed509a328d', 'asset_id': '62697312879578878537492465609249634498018844363287127652537828808816942160117', 'side': 'BUY', 'original_size': '5', '

In [ ]:
# Execute order
response = executor.execute_signal(signal)
print("\nLimit Order Response:")
print(response)



# Cancel all
executor.cancel_all_orders()
print("\nOpen Orders After Cancel:")
print(executor.client.get_open_orders())

Placing LIMIT order: BUY 5.0 of 29698297579308527847537871762106397437012761209817449145690502363235623683398 at 0.05
Error in create_and_post_order: Invalid order inputs

Limit Order Response:
None

Open Orders:
[]
Cancelling all orders.

Open Orders After Cancel:
[]
